This mock data uses a fixed global parameter + random draws for the age and theta parameters.

13) MOCK DATASETS (don't use the AM207 ones):

    a) Use some ISM parameters that represent different components of the ISM e.g. leave ISM parameters at their prior values and each time vary one by 2 sigma. one bursty SFR environment (SFR - 2sigma), one leaky (x_out + 2), one low SFE environment. And then see how well you can retrieve those. You will also see, that the ISM parameters are correlated (the signal will be redistributed into ather ISM parameters). I suggest it this way because in a paper you can plot the different tracks, name them (bursty, leaky etc) and plot the mock star abundances at specific times. And then you can show how well you can retrieve them. Of course you will have to add observational uncertainties to your predicted abundances. You can test with very low observational uncertainties, but a fair test would be to use typical observational uncertainties. This doesn't need any error parameters per element.
    
    b) In a last step you will need to do the inference with a wrong yield set as well (only then the error parameters per element will also be needed).



In [1]:
%pylab inline
import os
os.chdir('/home/oliver/ChempyMulti/')
from Chempy.parameter import ModelParameters
from Chempy.cem_function import single_timestep_chempy
a=ModelParameters()

Populating the interactive namespace from numpy and matplotlib


First we choose some SSP parameters:

In [43]:
from scipy.stats import norm
global_params = norm.rvs(loc=a.p0[:2],scale=[0.3,0.3]).round(2)

In [44]:
global_params

array([-2.52, -2.57])

In [185]:
# Use 5 time datapoints
N_stars = 200
times = np.random.uniform(2,12,N_stars)

log_SFR_lim = 0.29402

ism_priors = a.p0[2:]
ism_prior_widths = [0.3,0.1,0.1]

ism_params = [np.random.normal(loc=ism_priors[i],scale=ism_prior_widths[i],size=N_stars) for i in range(len(ism_priors))]
ism_params = np.asarray(ism_params).T.reshape(N_stars,len(ism_priors))

from scipy.stats import truncnorm
ism_params[:,1]=truncnorm.rvs((log_SFR_lim-ism_priors[1])/ism_prior_widths[1],np.inf,
                              loc=ism_priors[1],scale=ism_prior_widths[1],size=N_stars)

In [190]:
full_params = np.zeros([len(ism_params),6])
for i in range(len(full_params)):
    full_params[i]=np.asarray(list(global_params)+list(ism_params[i])+[times[i]])

In [191]:
out=np.inf
i=0
while out==np.inf:
    out=single_timestep_chempy((full_params[i],a))
    i+=1
_,chempy_els=out

In [192]:

def runner(par):
    """Function to compute the Chempy predictions for each parameter set"""
    b=ModelParameters()
    output=single_timestep_chempy((par,b))
    if type(output)==float:
        if output==np.inf:
            del b
        outs=np.zeros(len(chempy_els))
    else: 
        abun=output[0]
        del b;
        outs=abun
    return outs


In [193]:
mock_abundances=np.asarray([runner(par) for par in full_params])

In [201]:
good_id=np.where(mock_abundances[:,0]!=0)
mock_abundances2=mock_abundances[good_id]
times2=times[good_id]
full_params2=full_params[good_id]

In [202]:
len(full_params2)

200

Use observational errors as solar errors here:

In [203]:
sol_dat = np.load('Chempy/input/stars/Proto-sun_all.npy')
sol_names = sol_dat.dtype.names
all_sol_errors = sol_dat[1]

In [204]:
obs_errors = np.zeros(len(a.initial_neural_names))
for i,el in enumerate(a.initial_neural_names):
    for j,el_sol in enumerate(sol_names):
        if str(el)==str(sol_names[j]):
            obs_errors[i]=all_sol_errors[j]
            break
        if j==len(sol_names)-1:
            print('Element %s not found'%el)
mock_ordered_abundances = np.zeros([len(mock_abundances2),len(a.initial_neural_names)])        
for i,el in enumerate(a.initial_neural_names):
    for j,el_chempy in enumerate(chempy_els):
        if str(el)==str(chempy_els[j]):
            mock_ordered_abundances[:,i]=mock_abundances2[:,j]
            break
        if j==len(chempy_els)-1:
            print('Element %s not found'%el)
        

In [205]:
mock_errors = np.vstack([obs_errors for _ in range(len(mock_abundances2))])

In [234]:
from scipy.stats import norm
time_error = (0.05*times2).round(2)
time_mean = norm.rvs(loc=times2,scale=time_error).round(2)
#all_time_means = np.zeros(len(full_params2))
#all_time_stds = np.zeros(len(full_params2))
#for i in range(len(full_params2)):
#    all_time_means[i]=time_mean[i%len(times2)]
#    all_time_stds[i] = time_error[i%len(times2)]

In [238]:
np.savez('ChempyMultiData/MockParamsRandom.npz',params=full_params2,abundances=mock_ordered_abundances,
        time_mean = time_mean, time_error = time_error, abundance_error = mock_errors,elements=a.initial_neural_names)